In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

In [2]:
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter

In [3]:
## Reorder target contour
def reorder(h):
    h = h.reshape((4,2))

    
    hnew = np.zeros((4,2), dtype=np.float32)
    
    add = h.sum(axis=1)
    hnew[3] = h[np.argmin(add)]
    hnew[1] = h[np.argmax(add)]
    
    diff = np.diff(h, axis=1)
    hnew[0] = h[np.argmin(diff)]
    hnew[2] = h[np.argmax(diff)]
    
    return hnew

In [6]:
#img_path is path of the image needed to 
def convert_img_text(img_path):
    img = cv2.imread(img_path)
    print(img.shape)
    img = cv2.resize(img, (1500, 800))
    
    ## Image Blurring
    orig = img.copy()
    gray = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), 0)
    
    ## Edge Detection
    edged = cv2.Canny(blurred, 0, 50)
    orig_edged = edged.copy()
    
    
    ## Contours Extraction
    _, contours, _ = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    contours = sorted(contours, reverse=True, key=cv2.contourArea)
    
    ## Best Contour Selection
    for c in contours:
        p = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02*p, True)
    
        if len(approx) == 4:
            target = approx
            break
            
    #reordering points        
    reorderd = reorder(target)
    
    ## Project to a fixed size screen
    input_represent = reorderd
    output_map = np.float32([[0,0],[800,0],[800,800],[0,800]])
    
    M = cv2.getPerspectiveTransform(input_represent, output_map)
    ans = cv2.warpPerspective(orig, M, (800,800))
    
    
    #blurring again to increase readability
    ans2 = cv2.cvtColor(ans, cv2.COLOR_BGR2GRAY)
    temp = cv2.GaussianBlur(ans2, (1,1), 0)
    now = cv2.cvtColor(temp, cv2.COLOR_GRAY2BGR)
    
    #converting into text
    img=Image.fromarray(now)
    img.save("refined_receipt.jpg")
    text = pytesseract.image_to_string(Image.open("refined_receipt.jpg"))
    print(text)
    

In [7]:
convert_img_text('./receipt.jpg')

(2448, 3264, 3)
WHﬁLE
WHFOODS

UHDLE FOODS HﬂRKET — UESTPDRT.CT 05880
399 POST RD NEST — (203) 227—6858

 

365 BRCON LS NF 4 99
365 BHCDN LS NP 4.99
365 BQCON LS NP 4 99
365 BHCON LS NF 4 99

BROTH CHIC NP 2 19

FLOUR RLNOND NP 1!.99

CHKN BRST BNLSS SK NF 18 80
HEAVY CRERH NP 3.39

BHLSHC REDUCT NP 6.49

BEEF ERND 85/15 NP 5.01
JUICE CUP CRSHEU L NP 8.99
DOCS PINT DRGHNIC NP 14.49

HNY HLHDND BUTTER NP 9.99

lI-l Yﬁx .00 BBL ‘01.33

.“.l'ttlilti

‘nn'nﬂ-n-n-n-nmﬂﬂwﬂ
